In [1]:
r"""
Perform all-sky scans with no spatial prior information.
Record the TS (and number of true and fitted events around the
best fit location) for each pixel, creating a background-only TSD
at each point in the sky.
Combine this map with GBM healpix maps or grab a single pixel
for non-GBM bursts.
"""

'\nPerform all-sky scans with no spatial prior information.\nRecord the TS (and number of true and fitted events around the\nbest fit location) for each pixel, creating a background-only TSD\nat each point in the sky.\nCombine this map with GBM healpix maps or grab a single pixel\nfor non-GBM bursts.\n'

In [2]:
import os
import sys
import numpy as np
import healpy as hp
import histlite as hl
import csky as cy
import pandas as pd
from scipy import sparse

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as colors
%matplotlib inline
# %matplotlib notebook
from glob import glob
timer = cy.timing.Timer()
time = timer.time

###### Local Import ######
import SETTING
paths = SETTING.PATH()
print(paths)
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR

from utils import *


#### Working at GT ####

USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/version-002-p04
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [3]:
tw_batchSize_map = {
                 10:10000,
                 25:10000,
                 50:10000,
                 100:10000,
                 250:10000,
                 500:10000
                }

In [4]:
import argparse

######################### CONFIGURE ARGUEMENTS #############################
# p = argparse.ArgumentParser(description="Background Trials",
#                             formatter_class=argparse.RawTextHelpFormatter)
# p.add_argument("--grb_name", default="GRB190612A", type=str, help="Name of one GRB")
# p.add_argument("--batchNtrials", default=100, type=int, help="Number of trials in this batch")
# p.add_argument("--batchIndex", default=0, type=int, help="Index of current batch")
# p.add_argument("--tw_in_second", default=10, type=int, help="Length of the time window in seconds")
# p.add_argument("--concat", default=0, type=int, help="True(1) for the last batchIndex. used to clear up everything")
# p.add_argument("--totalNtrials", default=100000000, type=int, help="Number of total trials (1e8)")
# p.add_argument("--ncpu", default=1, type=int, help="Number of CPU to give Csky")
# p.add_argument("--mode", default="production", type=str, help="Mode: production or testing")
# args = p.parse_args()
###########################################################################

### testing on jupyter ###
class args:
    grb_name = "GRB180423A"    # real healpix example
    # grb_name = "GRB190611B"    # fake healpix example
    batchNtrials = 40
    batchIndex = 0
    tw_in_second = 10
    concat=0
    totalNtrials = 40
    ncpu = 4
    mode = "testing"
##########################

In [5]:
if tw_batchSize_map[args.tw_in_second] != args.batchNtrials and args.mode=="prodection":
    raise Exception("You are not using the recommended batchNtrials wrt this tw_in_second!")

In [6]:
print("\n===== Loading GRB list =====\n") 
if args.mode != "production":
    ANA_DIR = ANA_DIR + "/test"
# All times in days, all angles in degrees
try:
    df = pd.read_pickle(DATA_DIR+"/grbwebgbm/grbweb_gbm_noHeaplix.pkl")
except:
    raise Exception("Cannot pd.reade_picle() the grbweb_gbm_noHeaplix.pkl.\n")

print("\n...Done\n")
print("\n===== Loading healpix of {}=====\n".format(args.grb_name))   
try:
    healpix = np.load(DATA_DIR+"/grbwebgbm/healpix/{}_healpix_nside64.npy".format(args.grb_name))
    # healpix can contain negative values: faults due to Fermi-GBM
    healpix = np.maximum(healpix,0)
    ########## healpix reduce ##########
    healpix[healpix <= isf_healpix(healpix, q=0.95)] = 0
except:
    raise Exception("Cannot load the healpix for grb: {}\n".format(args.grb_name))
    
grb_row = df.loc[df['grb_name'] == args.grb_name]
tw = args.tw_in_second/86400.
tw_start = grb_row.t_center - 0.5*tw
ra = grb_row.ra
dec = grb_row.dec
print("\n...Done\n")


===== Loading GRB list =====


...Done


===== Loading healpix of GRB180423A=====


...Done



In [7]:
print("\n===== Setting up csky =====\n")
data_dir = ICDATA_DIR
data_filenames = sorted(glob(data_dir + '/IC86_20*.data_with_angErr.npy'))
sig_filenames = sorted(glob(data_dir + '/IC86_2012.nu*_merged_with_angErr.npy'))
grl_filenames = sorted(glob(data_dir + '/GRL/IC86_20*.data.npy'))

################ energy lower bound #############
min_log_e = np.log10(30)
#################################################
bins_sindec = np.linspace(-1, 1, 25+1)  
bins_logenergy = np.linspace(min_log_e, 5, 30+1)

data = [np.load(data_filename) for data_filename in data_filenames]
data = np.hstack(data)
sig = [np.load(sig_filename) for sig_filename in sig_filenames]
sig = np.hstack(sig)
grl = [np.load(grl_filename) for grl_filename in grl_filenames]
grl = np.hstack(grl)
if min_log_e is not None:
    data_mask = data['logE'] > min_log_e
    data = data[data_mask]
    sig_mask = sig['logE'] > min_log_e
    sig = sig[sig_mask]
    
dataset_spec = cy.selections.CustomDataSpecs.CustomDataSpec(data, sig, np.sum(grl['livetime']),
                                                     sindec_bins=bins_sindec,
                                                     logenergy_bins=bins_logenergy,
                                                     grl=grl, key='greco_v2.4', cascades=True)

ana_dir = cy.utils.ensure_dir(ANA_DIR)
ana = cy.get_analysis(cy.selections.repo, dataset_spec, dir=ana_dir, load_sig=True)    




===== Setting up csky =====

Setting up Analysis for:
greco_v2.4
Setting up greco_v2.4...
Energy PDF Ratio Model...
  * gamma = 4.0000 ...
Signal Acceptance Model...
  * gamma = 4.0000 ...
Done.


In [8]:
conf = {
    'ana': ana,
    #### llh basics: csky.conf
    'space': 'ps', # ps/fitps/template/prior
    'time': 'transient', # utf/lc/transient
    'energy': 'customflux', # fit/customflux
    'flux': cy.hyp.PowerLawFlux(2),
    #### inj.py - prior has some duplications against space's prior
    'sig': 'transient', # ps/tw/lc/transient/template/prior
    'full_sky': True,
    'extended': True,
    'mp_cpus': args.ncpu
    }

cy.CONF.update(conf)

In [9]:
print("\n===== Generating seeds for current batch =====\n")   
src = cy.utils.Sources(
    ra=ra,
    dec=dec,
    deg=True,
    mjd=tw_start, 
    sigma_t=np.zeros_like(tw), 
    t_100=tw,  # in days
    # prior=[hl.heal.HealHist(healpix)],
    name=args.grb_name
)
sstr = cy.get_sky_scan_trial_runner(conf=cy.CONF
                                    ,nside=64
                                    ,src_tr=src)

rng=np.random.default_rng(abs(java_hash(src.name[0])))
seeds = rng.integers(int(1e9), size=int(2e8))[args.batchNtrials*args.batchIndex: args.batchNtrials*(args.batchIndex + 1)]
print("\n...Done\n")


===== Generating seeds for current batch =====


...Done



In [10]:
all_pixel_TS = sparse.lil_matrix((len(seeds), hp.nside2npix(64)), dtype=float)
with time("allsky scramble scan"):
    for no_trial, seed in enumerate(seeds):
        if no_trial % (len(seeds) // 10) == 0:
            print("Working on no_trial: {} \n".format(no_trial))
        # scan (3,49152): -log10p, TS, ns
        scan = sstr.get_one_scan(n_sig=0
                                 , poisson=False
                                 , seed=seed
                                 , TRUTH=False
                                 , mp_cpus=args.ncpu
                                 , logging=False)
        all_pixel_TS[no_trial] = scan[1]

Working on no_trial: 0 

Working on no_trial: 4 

Working on no_trial: 8 

Working on no_trial: 12 

Working on no_trial: 16 

Working on no_trial: 20 

Working on no_trial: 24 

Working on no_trial: 28 

Working on no_trial: 32 

Working on no_trial: 36 


0:09:50.948551 elapsed.


In [11]:
with time("Saving...."):
    hp_sparse = all_pixel_TS.tocsr()
    output_folder = cy.utils.ensure_dir(ANA_DIR+"/allsky_scan/no_prior_versatile/{}/tw{}".format(args.grb_name, args.tw_in_second))
    sparse.save_npz(output_folder+"/{}_batchSize{}_batchIndex{}_tw{}.npz".format(args.grb_name, 
                                                                    args.batchNtrials, 
                                                                    args.batchIndex, 
                                                                    args.tw_in_second)
                    ,hp_sparse)
print("######## Done. ###########")



0:00:00.004628 elapsed.
######## Done. ###########
